In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import sklearn
sklearn.set_config(transform_output="pandas")
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import GridSearchCV, KFold
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_log_error
from sklearn.linear_model import LinearRegression
import optuna
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso
from sklearn.preprocessing import FunctionTransformer
import joblib


### Подгружаем датасет

In [7]:
train = pd.read_csv('/Users/polyakovk/ds_bootcamp/House_prices_kaggle/train.csv', index_col='Id')
test = pd.read_csv('/Users/polyakovk/ds_bootcamp/House_prices_kaggle/test.csv', index_col='Id')

### Объединяем в один

In [8]:
df = [train, test]
df = pd.concat(df)

### Отделяем целевую переменную

In [9]:
X, y = df.drop('SalePrice',axis=1), df['SalePrice']

### Избавляемся от пропусков в данных
выводим числовые и категориальные признаки  
там, где в признаках значение nan обусловлено отсутствием меняем на значение "Unknown"  
там, где пропуски случайны, заполняем модой для категориальных и медианой для числовых

In [10]:
numeric_features = X.select_dtypes(include=['number']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
print(list(categorical_features))

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [11]:
# Заполнение пропусков в категориальных   
replace_nan_col = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu',
                   'GarageType','GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']  # меняем значение nan на отсутствует
fill_mode_col = ['Alley', 'Electrical', ] # заполняем модой

median_impute_columns = numeric_features


# Удаление столбцов и заполнение пропусков в категориальных
imputer = ColumnTransformer(
    transformers = [
        ('replace', SimpleImputer(strategy='constant', fill_value='Unknown'), replace_nan_col),
        ('fill_mode', SimpleImputer(strategy='most_frequent'), fill_mode_col),
        ('median_imputer', SimpleImputer(strategy='median'), median_impute_columns),
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough' 
)

### Feature ingenering

In [12]:
def create_features(df):
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['OverallGrade'] = df['OverallQual'] * df['OverallCond']
    df['AgeAtSale'] = df['YrSold'] - df['YearBuilt']
    df['TotalBathrooms'] = df['FullBath'] + df['HalfBath']*0.5 + df['BsmtFullBath'] + df['BsmtHalfBath']*0.5
    
    
    # Удаление старых признаков, которые были использованы для создания новых
    drop_columns = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'OverallQual', 'OverallCond', 
                    'YearBuilt', 'YrSold', 'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 'Alley', 'MiscFeature']
    df = df.drop(columns=drop_columns)
    
    return df

drop_columns = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'OverallQual', 'OverallCond', 
                    'YearBuilt', 'YrSold', 'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 'Alley', 'MiscFeature',
                    'GarageQual', 'GarageCond']
new_feature = ['TotalSF', 'OverallGrade', 'AgeAtSale', 'TotalBathrooms']


# Создание пользовательского трансформера
feature_creator = FunctionTransformer(create_features, validate=False)

### Кодирование и нормировка  
one-hot - для признаков с небошим кол-вом значений  
ordinal - для признаков явно указывающих на градацию  
cat-boost - для признаков с большим количеством не связанных значений  
StandardScaler - для номировки числовых данных

In [13]:
numeric_features = numeric_features + new_feature
numeric_features = [item for item in numeric_features if item not in drop_columns]

one_hot_encoding_columns = ['Street', 'CentralAir', 'PavedDrive', ]
ordinal_encoding_columns = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC',
                            'Fence', 'LandSlope', 'ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual', ]
catboost_ecncoding_columns = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'GarageType', 'GarageFinish', 'Electrical','MSZoning', 'LotShape',
                              'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 
                              'HouseStyle', 'RoofStyle', 'RoofMatl','Exterior1st', 'Exterior2nd', 'Foundation', 'Heating', 'Functional', 'SaleType', 'SaleCondition']


scaler_encoder = ColumnTransformer(
    [
        ('one_hot_encoding', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), one_hot_encoding_columns),
        ('ordinal_encoding', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_encoding_columns),
        ('catboost_encoding_columns', CatBoostEncoder(), catboost_ecncoding_columns),
        ('scaler', StandardScaler(), numeric_features),
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough' 
)

### Собираем препроцессор

In [14]:
preprocessor = Pipeline(
    [
        ('imputer', imputer),
        ('feature_creator', feature_creator),
        ('scaler_encoder', scaler_encoder)
    ]
)

### Делим датафрейм в изначальных пропорциях и логарифмируем целевую переменную

In [15]:
X_train = pd.DataFrame(X[:1460])
test  = pd.DataFrame(X[1460:])
y_train = pd.Series(y[:1460])
y_train = np.log1p(y_train)

### Дополнительное разбиение train выборки на train и valid для тестов на размеченных данных

In [16]:
# X_train1, X_valid, Y_train1, Y_valid = train_test_split(X_train, y_train, train_size=0.8, test_size=0.2,random_state=42)

### Подбор гиперпараметров через Optuna  
Для подбора параметров, необходимо обучить и трансформировать данные через препроцессор и передать в оптюну  
затем записать лучшие параметры и передать в модель, перезапустив весь код, без транформа в препроцессор

In [31]:
# X_train = preprocessor.fit_transform(X_train, y_train)

In [17]:
# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
#         'depth': trial.suggest_int('depth', 3, 10),
#         'subsample': trial.suggest_uniform('subsample', 0.6, 0.95),
#         'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.6, 0.95),
#         'random_state': 42,
#         'loss_function': 'RMSE',
#         'logging_level': 'Silent'
#     }
    
#     model = CatBoostRegressor(**params)
#     cv = KFold(n_splits=5, random_state=42, shuffle=True)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_mean_squared_log_error')
#     rmsle = np.sqrt(-scores.mean())  # Конвертация в RMSLE
    
#     return rmsle

# # Создание и запуск оптимизации с помощью Optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# best_params = study.best_params
# best_rmsle = study.best_value

# print(f"Лучшие параметры: {best_params}")
# print(f"Лучшее значение RMSLE: {best_rmsle}")

# Лучшие параметры из оптюны

In [22]:
# старые ИСПОЛЬЗОВАТЬ ЭТИ
best_params = {'n_estimators': 800, 'learning_rate': 0.050939168701603, 'depth': 5, 'subsample': 0.8217157756680307, 'colsample_bylevel': 0.6002185179366489}

In [19]:
# # новые
# best_params = {'n_estimators': 900, 'learning_rate': 0.032881543434917726, 'depth': 6, 'subsample': 0.7790632706804598, 'colsample_bylevel': 0.9034758823706623}

### Собираем в пайплайн препроцессор и модель используя лучшие параметры из Optuna   
Обучаем модель на train части выборки

In [23]:
CB = CatBoostRegressor(**best_params)

ml_pipeline_CB = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', CB)
    ]
)

ml_pipeline_CB.fit(X_train, y_train)

0:	learn: 0.3860310	total: 60.5ms	remaining: 48.3s
1:	learn: 0.3736647	total: 61.9ms	remaining: 24.7s
2:	learn: 0.3619068	total: 63.3ms	remaining: 16.8s
3:	learn: 0.3510050	total: 64.6ms	remaining: 12.9s
4:	learn: 0.3418292	total: 66ms	remaining: 10.5s
5:	learn: 0.3309604	total: 67ms	remaining: 8.86s
6:	learn: 0.3217859	total: 68ms	remaining: 7.7s
7:	learn: 0.3112638	total: 68.9ms	remaining: 6.82s
8:	learn: 0.3023124	total: 70ms	remaining: 6.15s
9:	learn: 0.2935731	total: 71ms	remaining: 5.61s
10:	learn: 0.2853201	total: 72.3ms	remaining: 5.18s
11:	learn: 0.2771047	total: 73.6ms	remaining: 4.83s
12:	learn: 0.2693552	total: 74.9ms	remaining: 4.54s
13:	learn: 0.2621020	total: 76ms	remaining: 4.27s
14:	learn: 0.2561453	total: 77ms	remaining: 4.03s
15:	learn: 0.2492491	total: 78.2ms	remaining: 3.83s
16:	learn: 0.2436322	total: 79.8ms	remaining: 3.67s
17:	learn: 0.2377933	total: 80.9ms	remaining: 3.51s
18:	learn: 0.2322306	total: 81.8ms	remaining: 3.36s
19:	learn: 0.2269811	total: 82.7ms	re

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('replace',
                                                                   SimpleImputer(fill_value='Unknown',
                                                                                 strategy='constant'),
                                                                   ['MasVnrType',
                                                                    'BsmtQual',
                                                                    'BsmtCond',
                                                                    'BsmtExposure',
                                                                    'BsmtFinType1',
                                                                    'BsmtFinType2',
                                                                    'FireplaceQu',
                                                                    'GarageType',
                                                                    'GarageFinish',
                                                                    'GarageQual',
                                                                    'GarageCond',
                                                                    'PoolQC',
                                                                    'Fence',
                                                                    'M...
                                                                    'BedroomAbvGr',
                                                                    'KitchenAbvGr',
                                                                    'TotRmsAbvGrd',
                                                                    'Fireplaces',
                                                                    'GarageYrBlt',
                                                                    'GarageCars',
                                                                    'GarageArea',
                                                                    'WoodDeckSF',
                                                                    'OpenPorchSF',
                                                                    'EnclosedPorch',
                                                                    '3SsnPorch',
                                                                    'ScreenPorch',
                                                                    'PoolArea',
                                                                    'MiscVal',
                                                                    'MoSold',
                                                                    'TotalSF',
                                                                    'OverallGrade',
                                                                    'AgeAtSale',
                                                                    'TotalBathrooms'])],
                                                    verbose_feature_names_out=False))])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x15a631520>)])

### Строим предсказание на тестовой части выборки и записываем в файл Submission.csv

In [24]:
# y_pred = ml_pipeline_CB.predict(X_valid)
# rmlse = root_mean_squared_log_error(Y_valid, y_pred)
# y_pred = np.expm1(y_pred)
# rmlse
# print(f"RMSLE для CatBoost: {rmlse}")

In [25]:
y_pred = ml_pipeline_CB.predict(test)
y_pred = np.expm1(y_pred)
submission_cb = pd.DataFrame({'Id': test.index, 'SalePrice': y_pred})


In [26]:
submission_cb.to_csv('submission.csv', index=False)

In [27]:
joblib.dump(ml_pipeline_CB, 'model_pipeline.pkl')

['model_pipeline.pkl']